# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_b
　　セッション1のAさんのスケーリングデータ　→　data1_b_label_50

In [ ]:
%time
#sensor_dataの読み込み

# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

# data1_b_label_50 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_50.csv')
# data1_b_label_60 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_60.csv')
# data1_b_label_70 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_70.csv')
# data1_b_label_80 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_80.csv')

data2_b_label_50 = pd.read_csv('emotion_label/session2/B/emotion_B_20201111_2_50.csv')
data2_b_label_60 = pd.read_csv('emotion_label/session2/B/emotion_B_20201111_2_60.csv')
data2_b_label_70 = pd.read_csv('emotion_label/session2/B/emotion_B_20201111_2_70.csv')
data2_b_label_80 = pd.read_csv('emotion_label/session2/B/emotion_B_20201111_2_80.csv')

## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [ ]:
data2_b_label_80

In [ ]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

# data1_b_label_50 = dropna(data1_b_label_50)
# data1_b_label_60 = dropna(data1_b_label_60)
# data1_b_label_70 = dropna(data1_b_label_70)
# data1_b_label_80 = dropna(data1_b_label_80)

data2_b_label_50 = dropna(data2_b_label_50)
data2_b_label_60 = dropna(data2_b_label_60)
data2_b_label_70 = dropna(data2_b_label_70)
data2_b_label_80 = dropna(data2_b_label_80)

In [ ]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=0)
    return a

# data1_b_label_50 = dropna(data1_b_label_50)
# data1_b_label_60 = dropna(data1_b_label_60)
# data1_b_label_70 = dropna(data1_b_label_70)
# data1_b_label_80 = dropna(data1_b_label_80)

data2_b_label_50 = dropna(data2_b_label_50)
data2_b_label_60 = dropna(data2_b_label_60)
data2_b_label_70 = dropna(data2_b_label_70)
data2_b_label_80 = dropna(data2_b_label_80)

In [55]:
data2_b_label_80

,start_time,finish_time,emotion_level
0,3:46,3:50,4
1,3:47,3:51,4
2,3:48,3:52,4
3,3:49,3:53,4
4,3:50,3:54,4
...,...,...,...
662,18:09,18:13,3
663,18:10,18:14,3
664,18:11,18:15,3
665,18:12,18:16,3


In [56]:
#生データの確認
display(data2_b)
display(data2_b_label_80)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,14:57:33.415,9036,-574,5561,114,37,616
1,ags,14:57:33.420,9104,-693,5500,71,-17,652
2,ags,14:57:33.425,9092,-871,5583,132,-151,530
3,ags,14:57:33.430,9051,-710,5417,144,-285,268
4,ags,14:57:33.435,8677,-420,5197,175,-334,-61
...,...,...,...,...,...,...,...,...
494494,ags,15:38:46.020,9663,-76,2788,-81,257,366
494495,ags,15:38:46.025,9698,187,2736,-93,178,427
494496,ags,15:38:46.030,9861,-1,2800,-20,214,475
494497,ags,15:38:46.035,9951,-152,2885,17,257,445


,start_time,finish_time,emotion_level
0,3:46,3:50,4
1,3:47,3:51,4
2,3:48,3:52,4
3,3:49,3:53,4
4,3:50,3:54,4
...,...,...,...
662,18:09,18:13,3
663,18:10,18:14,3
664,18:11,18:15,3
665,18:12,18:16,3


In [57]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
# norm_order(data1_b)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
norm_order(data2_b)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [58]:
data2_b

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,14:57:33.415,9036,-574,5561,114,37,616,10625.605536,627.551592
1,ags,14:57:33.420,9104,-693,5500,71,-17,652,10658.942959,656.074691
2,ags,14:57:33.425,9092,-871,5583,132,-151,530,10704.811722,566.678921
3,ags,14:57:33.430,9051,-710,5417,144,-285,268,10572.066496,416.875281
4,ags,14:57:33.435,8677,-420,5197,175,-334,-61,10123.020202,381.971203
...,...,...,...,...,...,...,...,...,...,...
494494,ags,15:38:46.020,9663,-76,2788,-81,257,366,10057.449428,454.495325
494495,ags,15:38:46.025,9698,187,2736,-93,178,427,10078.287007,471.870745
494496,ags,15:38:46.030,9861,-1,2800,-20,214,475,10250.820553,521.364556
494497,ags,15:38:46.035,9951,-152,2885,17,257,445,10361.888341,514.162426


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [59]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

# datetime_order(data1_a)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

# datetime_order(data2_a)
datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [60]:
# data1_b_label_50['start_time'] = pd.to_datetime(data1_b_label_50['start_time'], format='%M:%S')
# data1_b_label_50['finish_time'] = pd.to_datetime(data1_b_label_50['finish_time'], format='%M:%S')

In [61]:
# data1_b_label_60['start_time'] = pd.to_datetime(data1_b_label_60['start_time'], format='%M:%S')
# data1_b_label_60['finish_time'] = pd.to_datetime(data1_b_label_60['finish_time'], format='%M:%S')

In [62]:
# data1_b_label_70['start_time'] = pd.to_datetime(data1_b_label_70['start_time'], format='%M:%S')
# data1_b_label_70['finish_time'] = pd.to_datetime(data1_b_label_70['finish_time'], format='%M:%S')

In [63]:
# data1_b_label_80['start_time'] = pd.to_datetime(data1_b_label_80['start_time'], format='%M:%S')
# data1_b_label_80['finish_time'] = pd.to_datetime(data1_b_label_80['finish_time'], format='%M:%S')

In [64]:
data2_b_label_50['start_time'] = pd.to_datetime(data2_b_label_50['start_time'], format='%M:%S')
data2_b_label_50['finish_time'] = pd.to_datetime(data2_b_label_50['finish_time'], format='%M:%S')

In [65]:
data2_b_label_60['start_time'] = pd.to_datetime(data2_b_label_60['start_time'], format='%M:%S')
data2_b_label_60['finish_time'] = pd.to_datetime(data2_b_label_60['finish_time'], format='%M:%S')

In [66]:
data2_b_label_70['start_time'] = pd.to_datetime(data2_b_label_70['start_time'], format='%M:%S')
data2_b_label_70['finish_time'] = pd.to_datetime(data2_b_label_70['finish_time'], format='%M:%S')

In [67]:
data2_b_label_80['start_time'] = pd.to_datetime(data2_b_label_80['start_time'], format='%M:%S')
data2_b_label_80['finish_time'] = pd.to_datetime(data2_b_label_80['finish_time'], format='%M:%S')

In [68]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_b)
display(data2_b_label_80)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:57:33.415,9036,-574,5561,114,37,616,10625.605536,627.551592
1,ags,2020-11-11 14:57:33.420,9104,-693,5500,71,-17,652,10658.942959,656.074691
2,ags,2020-11-11 14:57:33.425,9092,-871,5583,132,-151,530,10704.811722,566.678921
3,ags,2020-11-11 14:57:33.430,9051,-710,5417,144,-285,268,10572.066496,416.875281
4,ags,2020-11-11 14:57:33.435,8677,-420,5197,175,-334,-61,10123.020202,381.971203
...,...,...,...,...,...,...,...,...,...,...
494494,ags,2020-11-11 15:38:46.020,9663,-76,2788,-81,257,366,10057.449428,454.495325
494495,ags,2020-11-11 15:38:46.025,9698,187,2736,-93,178,427,10078.287007,471.870745
494496,ags,2020-11-11 15:38:46.030,9861,-1,2800,-20,214,475,10250.820553,521.364556
494497,ags,2020-11-11 15:38:46.035,9951,-152,2885,17,257,445,10361.888341,514.162426


,start_time,finish_time,emotion_level
0,1900-01-01 00:03:46,1900-01-01 00:03:50,4
1,1900-01-01 00:03:47,1900-01-01 00:03:51,4
2,1900-01-01 00:03:48,1900-01-01 00:03:52,4
3,1900-01-01 00:03:49,1900-01-01 00:03:53,4
4,1900-01-01 00:03:50,1900-01-01 00:03:54,4
...,...,...,...
662,1900-01-01 00:18:09,1900-01-01 00:18:13,3
663,1900-01-01 00:18:10,1900-01-01 00:18:14,3
664,1900-01-01 00:18:11,1900-01-01 00:18:15,3
665,1900-01-01 00:18:12,1900-01-01 00:18:16,3


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [69]:
# data1_b_label_50['start_time'] = data1_b_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data1_b_label_50['finish_time'] = data1_b_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [70]:
# data1_b_label_60['start_time'] = data1_b_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data1_b_label_60['finish_time'] = data1_b_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [71]:
# data1_b_label_70['start_time'] = data1_b_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data1_b_label_70['finish_time'] = data1_b_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [72]:
# data1_b_label_80['start_time'] = data1_b_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data1_b_label_80['finish_time'] = data1_b_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [73]:
data2_b_label_50['start_time'] = data2_b_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label_50['finish_time'] = data2_b_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [74]:
data2_b_label_60['start_time'] = data2_b_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label_60['finish_time'] = data2_b_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [75]:
data2_b_label_70['start_time'] = data2_b_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label_70['finish_time'] = data2_b_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [76]:
data2_b_label_80['start_time'] = data2_b_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label_80['finish_time'] = data2_b_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [77]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_b_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:30,2020-11-11 15:15:34,4
1,2020-11-11 15:15:31,2020-11-11 15:15:35,4
2,2020-11-11 15:15:32,2020-11-11 15:15:36,4
3,2020-11-11 15:15:33,2020-11-11 15:15:37,4
4,2020-11-11 15:15:34,2020-11-11 15:15:38,4
...,...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57,3
663,2020-11-11 15:29:54,2020-11-11 15:29:58,3
664,2020-11-11 15:29:55,2020-11-11 15:29:59,3
665,2020-11-11 15:29:56,2020-11-11 15:30:00,3


In [78]:
#datetime化したラベルデータを保存
# data1_b_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_b_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_b_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_b_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_b_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_b_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_b_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_b_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [79]:
#datetime化した加速度及び角速度データを保存
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [80]:
#時刻をインデックスに
# data1_b.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_a.set_index("時刻", inplace=True)
data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [81]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_b)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 14:57:33.415,ags,9036,-574,5561,114,37,616,10625.605536,627.551592
2020-11-11 14:57:33.420,ags,9104,-693,5500,71,-17,652,10658.942959,656.074691
2020-11-11 14:57:33.425,ags,9092,-871,5583,132,-151,530,10704.811722,566.678921
2020-11-11 14:57:33.430,ags,9051,-710,5417,144,-285,268,10572.066496,416.875281
2020-11-11 14:57:33.435,ags,8677,-420,5197,175,-334,-61,10123.020202,381.971203
...,...,...,...,...,...,...,...,...,...
2020-11-11 15:38:46.020,ags,9663,-76,2788,-81,257,366,10057.449428,454.495325
2020-11-11 15:38:46.025,ags,9698,187,2736,-93,178,427,10078.287007,471.870745
2020-11-11 15:38:46.030,ags,9861,-1,2800,-20,214,475,10250.820553,521.364556


In [82]:
# data1_b_label_50["start_time"] = data1_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data1_b_label_50["finish_time"] = data1_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [83]:
# data1_b_label_60["start_time"] = data1_b_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data1_b_label_60["finish_time"] = data1_b_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [84]:
# data1_b_label_70["start_time"] = data1_b_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data1_b_label_70["finish_time"] = data1_b_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [85]:
# data1_b_label_80["start_time"] = data1_b_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data1_b_label_80["finish_time"] = data1_b_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [86]:
data2_b_label_50["start_time"] = data2_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label_50["finish_time"] = data2_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [87]:
data2_b_label_60["start_time"] = data2_b_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label_60["finish_time"] = data2_b_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [88]:
data2_b_label_70["start_time"] = data2_b_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label_70["finish_time"] = data2_b_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [89]:
data2_b_label_80["start_time"] = data2_b_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label_80["finish_time"] = data2_b_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [90]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_b_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:30,2020-11-11 15:15:34,4
1,2020-11-11 15:15:31,2020-11-11 15:15:35,4
2,2020-11-11 15:15:32,2020-11-11 15:15:36,4
3,2020-11-11 15:15:33,2020-11-11 15:15:37,4
4,2020-11-11 15:15:34,2020-11-11 15:15:38,4
...,...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57,3
663,2020-11-11 15:29:54,2020-11-11 15:29:58,3
664,2020-11-11 15:29:55,2020-11-11 15:29:59,3
665,2020-11-11 15:29:56,2020-11-11 15:30:00,3


In [91]:
#startとfinishをリスト化
# data1_b_50_start_time = list(data1_b_label_50["start_time"])
# data1_b_50_finish_time = list(data1_b_label_50["finish_time"])

# data1_b_60_start_time = list(data1_b_label_60["start_time"])
# data1_b_60_finish_time = list(data1_b_label_60["finish_time"])

# data1_b_70_start_time = list(data1_b_label_70["start_time"])
# data1_b_70_finish_time = list(data1_b_label_70["finish_time"])

# data1_b_80_start_time = list(data1_b_label_80["start_time"])
# data1_b_80_finish_time = list(data1_b_label_80["finish_time"])

data2_b_50_start_time = list(data2_b_label_50["start_time"])
data2_b_50_finish_time = list(data2_b_label_50["finish_time"])

data2_b_60_start_time = list(data2_b_label_60["start_time"])
data2_b_60_finish_time = list(data2_b_label_60["finish_time"])

data2_b_70_start_time = list(data2_b_label_70["start_time"])
data2_b_70_finish_time = list(data2_b_label_70["finish_time"])

data2_b_80_start_time = list(data2_b_label_80["start_time"])
data2_b_80_finish_time = list(data2_b_label_80["finish_time"])

In [92]:
data2_b

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 14:57:33.415,ags,9036,-574,5561,114,37,616,10625.605536,627.551592
2020-11-11 14:57:33.420,ags,9104,-693,5500,71,-17,652,10658.942959,656.074691
2020-11-11 14:57:33.425,ags,9092,-871,5583,132,-151,530,10704.811722,566.678921
2020-11-11 14:57:33.430,ags,9051,-710,5417,144,-285,268,10572.066496,416.875281
2020-11-11 14:57:33.435,ags,8677,-420,5197,175,-334,-61,10123.020202,381.971203
...,...,...,...,...,...,...,...,...,...
2020-11-11 15:38:46.020,ags,9663,-76,2788,-81,257,366,10057.449428,454.495325
2020-11-11 15:38:46.025,ags,9698,187,2736,-93,178,427,10078.287007,471.870745
2020-11-11 15:38:46.030,ags,9861,-1,2800,-20,214,475,10250.820553,521.364556


In [93]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

# data1_b_record_1 = sensor_segmentation_order(data1_b, data1_b_50_start_time, data1_b_50_finish_time, data1_b_label_50)
# data1_b_record_2 = sensor_segmentation_order(data1_b, data1_b_60_start_time, data1_b_60_finish_time, data1_b_label_60)
# data1_b_record_3 = sensor_segmentation_order(data1_b, data1_b_70_start_time, data1_b_70_finish_time, data1_b_label_70)
# data1_b_record_4 = sensor_segmentation_order(data1_b, data1_b_80_start_time, data1_b_80_finish_time, data1_b_label_80)

data2_b_record_1 = sensor_segmentation_order(data2_b, data2_b_50_start_time, data2_b_50_finish_time, data2_b_label_50)
data2_b_record_2 = sensor_segmentation_order(data2_b, data2_b_60_start_time, data2_b_60_finish_time, data2_b_label_60)
data2_b_record_3 = sensor_segmentation_order(data2_b, data2_b_70_start_time, data2_b_70_finish_time, data2_b_label_70)
data2_b_record_4 = sensor_segmentation_order(data2_b, data2_b_80_start_time, data2_b_80_finish_time, data2_b_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [94]:
#加速度x,y,z,norm平均値
def acc_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]

    mean_list = pd.concat([acc_mean_list], axis=1, ignore_index=False)
    return mean_list

# data1_b_50_record_mean = acc_gyr_mean_order(data1_b_record_1, data1_b_label_50)
# data1_b_60_record_mean = acc_gyr_mean_order(data1_b_record_2, data1_b_label_60)
# data1_b_70_record_mean = acc_gyr_mean_order(data1_b_record_3, data1_b_label_70)
# data1_b_80_record_mean = acc_gyr_mean_order(data1_b_record_4, data1_b_label_80)

data2_b_50_record_mean = acc_mean_order(data2_b_record_1, data2_b_label_50)
data2_b_60_record_mean = acc_mean_order(data2_b_record_2, data2_b_label_60)
data2_b_70_record_mean = acc_mean_order(data2_b_record_3, data2_b_label_70)
data2_b_80_record_mean = acc_mean_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_mean
# b = data1_b_60_record_mean
# c = data1_b_70_record_mean
# d = data1_b_80_record_mean

# e = data2_b_50_record_mean
# f = data2_b_60_record_mean
# g = data2_b_70_record_mean
# h = data2_b_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [95]:
#加速度x,y,z,norm分散値
def acc_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    
    var_list = pd.concat([acc_var_list], axis=1, ignore_index=False)
    return var_list

# data1_b_50_record_var = acc_gyr_var_order(data1_b_record_1, data1_b_label_50)
# data1_b_60_record_var = acc_gyr_var_order(data1_b_record_2, data1_b_label_60)
# data1_b_70_record_var = acc_gyr_var_order(data1_b_record_3, data1_b_label_70)
# data1_b_80_record_var = acc_gyr_var_order(data1_b_record_4, data1_b_label_80)

data2_b_50_record_var = acc_var_order(data2_b_record_1, data2_b_label_50)
data2_b_60_record_var = acc_var_order(data2_b_record_2, data2_b_label_60)
data2_b_70_record_var = acc_var_order(data2_b_record_3, data2_b_label_70)
data2_b_80_record_var = acc_var_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_var
# b = data1_b_60_record_var
# c = data1_b_70_record_var
# d = data1_b_80_record_var

# e = data2_b_50_record_var
# f = data2_b_60_record_var
# g = data2_b_70_record_var
# h = data2_b_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [96]:
#加速度x,y,z,norm分散値
def acc_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    
    max_list = pd.concat([acc_max_list], axis=1, ignore_index=False)
    return max_list

# data1_b_50_record_max = acc_gyr_max_order(data1_b_record_1, data1_b_label_50)
# data1_b_60_record_max = acc_gyr_max_order(data1_b_record_2, data1_b_label_60)
# data1_b_70_record_max = acc_gyr_max_order(data1_b_record_3, data1_b_label_70)
# data1_b_80_record_max = acc_gyr_max_order(data1_b_record_4, data1_b_label_80)

data2_b_50_record_max = acc_max_order(data2_b_record_1, data2_b_label_50)
data2_b_60_record_max = acc_max_order(data2_b_record_2, data2_b_label_60)
data2_b_70_record_max = acc_max_order(data2_b_record_3, data2_b_label_70)
data2_b_80_record_max = acc_max_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_max
# b = data1_b_60_record_max
# c = data1_b_70_record_max
# d = data1_b_80_record_max

# e = data2_b_50_record_max
# f = data2_b_60_record_max
# g = data2_b_70_record_max
# h = data2_b_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [97]:
#加速度x,y,z,norm分散値
def acc_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    
    min_list = pd.concat([acc_min_list], axis=1, ignore_index=False)
    return min_list

# data1_b_50_record_min = acc_gyr_min_order(data1_b_record_1, data1_b_label_50)
# data1_b_60_record_min = acc_gyr_min_order(data1_b_record_2, data1_b_label_60)
# data1_b_70_record_min = acc_gyr_min_order(data1_b_record_3, data1_b_label_70)
# data1_b_80_record_min = acc_gyr_min_order(data1_b_record_4, data1_b_label_80)

data2_b_50_record_min = acc_min_order(data2_b_record_1, data2_b_label_50)
data2_b_60_record_min = acc_min_order(data2_b_record_2, data2_b_label_60)
data2_b_70_record_min = acc_min_order(data2_b_record_3, data2_b_label_70)
data2_b_80_record_min = acc_min_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_min
# b = data1_b_60_record_min
# c = data1_b_70_record_min
# d = data1_b_80_record_min

# e = data2_b_50_record_min
# f = data2_b_60_record_min
# g = data2_b_70_record_min
# h = data2_b_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [98]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max], axis=1, ignore_index=False)
    return b

# data1_b_50_record_fq_amp_max = fq_amp_max_order(data1_b_record_1, data1_b_label_50)
# data1_b_60_record_fq_amp_max = fq_amp_max_order(data1_b_record_2, data1_b_label_60)
# data1_b_70_record_fq_amp_max = fq_amp_max_order(data1_b_record_3, data1_b_label_70)
# data1_b_80_record_fq_amp_max = fq_amp_max_order(data1_b_record_4, data1_b_label_80)

data2_b_50_record_fq_amp_max = fq_amp_max_order(data2_b_record_1, data2_b_label_50)
data2_b_60_record_fq_amp_max = fq_amp_max_order(data2_b_record_2, data2_b_label_60)
data2_b_70_record_fq_amp_max = fq_amp_max_order(data2_b_record_3, data2_b_label_70)
data2_b_80_record_fq_amp_max = fq_amp_max_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_fq_amp_max
# b = data1_b_60_record_fq_amp_max
# c = data1_b_70_record_fq_amp_max
# d = data1_b_80_record_fq_amp_max

# e = data2_b_50_record_fq_amp_max
# f = data2_b_60_record_fq_amp_max
# g = data2_b_70_record_fq_amp_max
# h = data2_b_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [99]:
#全特徴量の行列の導出
# data1_b_50_record_all_feature = pd.concat([data1_b_50_record_mean, data1_b_50_record_var, data1_b_50_record_max, data1_b_50_record_min, data1_b_50_record_fq_amp_max], axis=1, ignore_index=False)
# data1_b_60_record_all_feature = pd.concat([data1_b_60_record_mean, data1_b_60_record_var, data1_b_60_record_max, data1_b_60_record_min, data1_b_60_record_fq_amp_max], axis=1, ignore_index=False)
# data1_b_70_record_all_feature = pd.concat([data1_b_70_record_mean, data1_b_70_record_var, data1_b_70_record_max, data1_b_70_record_min, data1_b_70_record_fq_amp_max], axis=1, ignore_index=False)
# data1_b_80_record_all_feature = pd.concat([data1_b_80_record_mean, data1_b_80_record_var, data1_b_80_record_max, data1_b_80_record_min, data1_b_80_record_fq_amp_max], axis=1, ignore_index=False)

data2_b_50_record_all_feature = pd.concat([data2_b_50_record_mean, data2_b_50_record_var, data2_b_50_record_max, data2_b_50_record_min, data2_b_50_record_fq_amp_max], axis=1, ignore_index=False)
data2_b_60_record_all_feature = pd.concat([data2_b_60_record_mean, data2_b_60_record_var, data2_b_60_record_max, data2_b_60_record_min, data2_b_60_record_fq_amp_max], axis=1, ignore_index=False)
data2_b_70_record_all_feature = pd.concat([data2_b_70_record_mean, data2_b_70_record_var, data2_b_70_record_max, data2_b_70_record_min, data2_b_70_record_fq_amp_max], axis=1, ignore_index=False)
data2_b_80_record_all_feature = pd.concat([data2_b_80_record_mean, data2_b_80_record_var, data2_b_80_record_max, data2_b_80_record_min, data2_b_80_record_fq_amp_max], axis=1, ignore_index=False)

In [100]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data2_b_80_record_all_feature

,accx_mean,accy_mean,accz_mean,accx_var,accy_var,accz_var,accx_max,accy_max,accz_max,accx_min,accy_min,accz_min,accn_max_fq,accn_max_peak
0,10254.309,463.512,-408.743,22558.381519,25334.803856,161229.896951,10847,895,742,9744,-20,-1757,17.612524,72.103750
1,10259.147,439.837,-339.496,25821.535391,28963.370431,172913.749984,10794,895,742,9744,-20,-1757,17.612524,51.250395
2,10263.035,450.146,-297.635,26485.223775,29802.692684,146525.327775,10816,895,683,9768,-20,-1540,4.305284,67.363613
3,10264.900,452.556,-228.331,25215.186000,25088.510864,128302.815439,10816,895,752,9768,-20,-1540,4.305284,82.347653
4,10264.243,464.621,-139.944,23955.353951,21768.005359,135603.590864,10816,895,752,9768,-20,-1540,4.696673,108.156118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,10271.765,86.960,-324.980,181459.649775,41769.212400,761050.285600,12439,1125,3444,8284,-886,-5268,4.305284,259.823134
663,10265.554,132.897,-363.220,146349.167084,53108.656391,725236.153600,12439,1125,3444,8284,-886,-5268,3.913894,339.025962
664,10267.061,191.674,-404.383,91678.371279,44079.669724,494076.604311,11470,881,2959,9273,-389,-1777,3.913894,252.719272
665,10263.345,225.737,-501.085,67391.975975,39316.481831,308810.597775,11470,817,1162,9380,-355,-1674,3.522505,192.303019


In [101]:
#各特徴量データの保存
# data1_b_50_record_all_feature.to_csv('feature/B/data1_b_50_record_all_feature.csv', index=False)
# data1_b_60_record_all_feature.to_csv('feature/B/data1_b_60_record_all_feature.csv', index=False)
# data1_b_70_record_all_feature.to_csv('feature/B/data1_b_70_record_all_feature.csv', index=False)
# data1_b_80_record_all_feature.to_csv('feature/B/data1_b_80_record_all_feature.csv', index=False)

data2_b_50_record_all_feature.to_csv('feature/B/data2_b_50_record_all_feature.csv', index=False)
data2_b_60_record_all_feature.to_csv('feature/B/data2_b_60_record_all_feature.csv', index=False)
data2_b_70_record_all_feature.to_csv('feature/B/data2_b_70_record_all_feature.csv', index=False)
data2_b_80_record_all_feature.to_csv('feature/B/data2_b_80_record_all_feature.csv', index=False)

In [102]:
data2_b_label_80

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:30,2020-11-11 15:15:34,4
1,2020-11-11 15:15:31,2020-11-11 15:15:35,4
2,2020-11-11 15:15:32,2020-11-11 15:15:36,4
3,2020-11-11 15:15:33,2020-11-11 15:15:37,4
4,2020-11-11 15:15:34,2020-11-11 15:15:38,4
...,...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57,3
663,2020-11-11 15:29:54,2020-11-11 15:29:58,3
664,2020-11-11 15:29:55,2020-11-11 15:29:59,3
665,2020-11-11 15:29:56,2020-11-11 15:30:00,3


In [103]:
# data1_b_label_50_time = data1_b_label_50.drop("emotion_level", axis=1)
# data1_b_label_60_time = data1_b_label_60.drop("emotion_level", axis=1)
# data1_b_label_70_time = data1_b_label_70.drop("emotion_level", axis=1)
# data1_b_label_80_time = data1_b_label_80.drop("emotion_level", axis=1)

data2_b_label_50_time = data2_b_label_50.drop("emotion_level", axis=1)
data2_b_label_60_time = data2_b_label_60.drop("emotion_level", axis=1)
data2_b_label_70_time = data2_b_label_70.drop("emotion_level", axis=1)
data2_b_label_80_time = data2_b_label_80.drop("emotion_level", axis=1)

In [104]:
data2_b_label_80_time

,start_time,finish_time
0,2020-11-11 15:15:30,2020-11-11 15:15:34
1,2020-11-11 15:15:31,2020-11-11 15:15:35
2,2020-11-11 15:15:32,2020-11-11 15:15:36
3,2020-11-11 15:15:33,2020-11-11 15:15:37
4,2020-11-11 15:15:34,2020-11-11 15:15:38
...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57
663,2020-11-11 15:29:54,2020-11-11 15:29:58
664,2020-11-11 15:29:55,2020-11-11 15:29:59
665,2020-11-11 15:29:56,2020-11-11 15:30:00


In [105]:
# data1_b_50_record_all_feature = pd.concat([data1_b_label_50_time, data1_b_50_record_all_feature], axis=1)
# data1_b_60_record_all_feature = pd.concat([data1_b_label_60_time, data1_b_60_record_all_feature], axis=1)
# data1_b_70_record_all_feature = pd.concat([data1_b_label_70_time, data1_b_70_record_all_feature], axis=1)
# data1_b_80_record_all_feature = pd.concat([data1_b_label_80_time, data1_b_80_record_all_feature], axis=1)

data2_b_50_record_all_feature = pd.concat([data2_b_label_50_time, data2_b_50_record_all_feature], axis=1)
data2_b_60_record_all_feature = pd.concat([data2_b_label_60_time, data2_b_60_record_all_feature], axis=1)
data2_b_70_record_all_feature = pd.concat([data2_b_label_70_time, data2_b_70_record_all_feature], axis=1)
data2_b_80_record_all_feature = pd.concat([data2_b_label_80_time, data2_b_80_record_all_feature], axis=1)

In [106]:
data2_b_80_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,accx_var,accy_var,accz_var,accx_max,accy_max,accz_max,accx_min,accy_min,accz_min,accn_max_fq,accn_max_peak
0,2020-11-11 15:15:30,2020-11-11 15:15:34,10254.309,463.512,-408.743,22558.381519,25334.803856,161229.896951,10847,895,742,9744,-20,-1757,17.612524,72.103750
1,2020-11-11 15:15:31,2020-11-11 15:15:35,10259.147,439.837,-339.496,25821.535391,28963.370431,172913.749984,10794,895,742,9744,-20,-1757,17.612524,51.250395
2,2020-11-11 15:15:32,2020-11-11 15:15:36,10263.035,450.146,-297.635,26485.223775,29802.692684,146525.327775,10816,895,683,9768,-20,-1540,4.305284,67.363613
3,2020-11-11 15:15:33,2020-11-11 15:15:37,10264.900,452.556,-228.331,25215.186000,25088.510864,128302.815439,10816,895,752,9768,-20,-1540,4.305284,82.347653
4,2020-11-11 15:15:34,2020-11-11 15:15:38,10264.243,464.621,-139.944,23955.353951,21768.005359,135603.590864,10816,895,752,9768,-20,-1540,4.696673,108.156118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57,10271.765,86.960,-324.980,181459.649775,41769.212400,761050.285600,12439,1125,3444,8284,-886,-5268,4.305284,259.823134
663,2020-11-11 15:29:54,2020-11-11 15:29:58,10265.554,132.897,-363.220,146349.167084,53108.656391,725236.153600,12439,1125,3444,8284,-886,-5268,3.913894,339.025962
664,2020-11-11 15:29:55,2020-11-11 15:29:59,10267.061,191.674,-404.383,91678.371279,44079.669724,494076.604311,11470,881,2959,9273,-389,-1777,3.913894,252.719272
665,2020-11-11 15:29:56,2020-11-11 15:30:00,10263.345,225.737,-501.085,67391.975975,39316.481831,308810.597775,11470,817,1162,9380,-355,-1674,3.522505,192.303019


In [107]:
# data1_b_50_record_all_feature.to_csv('feature/B/data1_b_50_record_all_feature.csv', index=False)
# data1_b_60_record_all_feature.to_csv('feature/B/data1_b_60_record_all_feature.csv', index=False)
# data1_b_70_record_all_feature.to_csv('feature/B/data1_b_70_record_all_feature.csv', index=False)
# data1_b_80_record_all_feature.to_csv('feature/B/data1_b_80_record_all_feature.csv', index=False)

data2_b_50_record_all_feature.to_csv('feature/B/data2_b_50_record_all_feature.csv', index=False)
data2_b_60_record_all_feature.to_csv('feature/B/data2_b_60_record_all_feature.csv', index=False)
data2_b_70_record_all_feature.to_csv('feature/B/data2_b_70_record_all_feature.csv', index=False)
data2_b_80_record_all_feature.to_csv('feature/B/data2_b_80_record_all_feature.csv', index=False)

In [108]:
data2_b_80_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,accx_var,accy_var,accz_var,accx_max,accy_max,accz_max,accx_min,accy_min,accz_min,accn_max_fq,accn_max_peak
0,2020-11-11 15:15:30,2020-11-11 15:15:34,10254.309,463.512,-408.743,22558.381519,25334.803856,161229.896951,10847,895,742,9744,-20,-1757,17.612524,72.103750
1,2020-11-11 15:15:31,2020-11-11 15:15:35,10259.147,439.837,-339.496,25821.535391,28963.370431,172913.749984,10794,895,742,9744,-20,-1757,17.612524,51.250395
2,2020-11-11 15:15:32,2020-11-11 15:15:36,10263.035,450.146,-297.635,26485.223775,29802.692684,146525.327775,10816,895,683,9768,-20,-1540,4.305284,67.363613
3,2020-11-11 15:15:33,2020-11-11 15:15:37,10264.900,452.556,-228.331,25215.186000,25088.510864,128302.815439,10816,895,752,9768,-20,-1540,4.305284,82.347653
4,2020-11-11 15:15:34,2020-11-11 15:15:38,10264.243,464.621,-139.944,23955.353951,21768.005359,135603.590864,10816,895,752,9768,-20,-1540,4.696673,108.156118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2020-11-11 15:29:53,2020-11-11 15:29:57,10271.765,86.960,-324.980,181459.649775,41769.212400,761050.285600,12439,1125,3444,8284,-886,-5268,4.305284,259.823134
663,2020-11-11 15:29:54,2020-11-11 15:29:58,10265.554,132.897,-363.220,146349.167084,53108.656391,725236.153600,12439,1125,3444,8284,-886,-5268,3.913894,339.025962
664,2020-11-11 15:29:55,2020-11-11 15:29:59,10267.061,191.674,-404.383,91678.371279,44079.669724,494076.604311,11470,881,2959,9273,-389,-1777,3.913894,252.719272
665,2020-11-11 15:29:56,2020-11-11 15:30:00,10263.345,225.737,-501.085,67391.975975,39316.481831,308810.597775,11470,817,1162,9380,-355,-1674,3.522505,192.303019
